# Multiple Outputs

In this chapter, you will build neural networks with multiple outputs, which can be used to solve regression problems with multiple targets. You will also build a model that solves a regression problem and a classification problem simultaneously.

# (1) Two-output models 

## Simple model with 2 outputs

In [ ]:
from keras.layers import Input, Concatenate, Dense
input_tensor = Input(shape=(1,))
output_tensor = Dense(2)(input_tensor)

<img src="image/Screenshot 2021-01-31 234418.png">

In [ ]:
from keras.models import Model
model = Model(input_tensor, output_tensor)
model.compile(optimizer='adam', loss='mean_absolute_error')

<img src="image/Screenshot 2021-01-31 234612.png">

## Fiting a model with 2 outputs

In [ ]:
games_tourney_train[['seed_diff', 'score_1', 'score_2']].head()

| | Seed_diff | Score_1 | Score_2 |
| :-: | :-: | :-: | :-: |
| 0 | -3 | 41 | 50 |
| 1 | 4 | 61 | 55 |
| 2 | 5 | 59 | 63 |
| 3 | 3 | 50 | 41 |
| 4 | 1 | 54 | 63 |

In [ ]:
X = games_tourney_train[['seed_diff']]
y = games_tourney_train[['score_1', 'score_2']]
model.fit(X, y, epochs=500)

## Inspecting a 2 output model

In [ ]:
model.get_weights()

In [ ]:
[array([[0.60714734]], dtype=float32), 
array([70.39491, 70.39306], dtype=float32)]

## Evaluating a model with 2 outputs

In [ ]:
X = games_tourney_test[['seed_diff']]
y = games_tourney_test[['score_1', 'score_2']]
model.evalute(X, y)

In [ ]:
11.528035634635021

# Exercise I: Simple two-output model

In this exercise, you will use the tournament data to build one model that makes two predictions: the scores of both teams in a given game. Your inputs will be the seed difference of the two teams, as well as the predicted score difference from the model you built in chapter 3.

The output from your model will be the predicted score for team 1 as well as team 2. This is called "multiple target regression": one model making more than one prediction.

### Instructions

- Create a single input layer with 2 columns.
- Connect this input to a Dense layer with 2 units.
- Create a model with `input_tensor` as the input and `output_tensor` as the output.
- Compile the model with `'adam'` as the optimizer and `'mean_absolute_error'` as the loss function.


In [ ]:
# Define the input
input_tensor = Input(shape=(2,))

# Define the output
output_tensor = Dense(2)(input_tensor)

# Create a model
model = Model(input_tensor, output_tensor)

# Compile the model
model.compile(optimizer='adam', loss='mean_absolute_error')

## Exercise II: Fit a model with two outputs

Now that you've defined your 2-output model, fit it to the tournament data. I've split the data into `games_tourney_train` and `games_tourney_test`, so use the training set to fit for now.

This model will use the pre-tournament seeds, as well as your pre-tournament predictions from the regular season model you built previously in this course.

As a reminder, this model will predict the scores of both teams.

### Instructions

- Fit the model to the `games_tourney_train` dataset using 100 epochs and a batch size of 16384.
- The input columns are `'seed_diff'`, and `'pred'`.
- The target columns are `'score_1'` and `'score_2'`.


In [ ]:
# Fit the model
model.fit(games_tourney_train[['seed_diff', 'pred']],
  		  games_tourney_train[['score_1', 'score_2']],
  		  verbose=True,
  		  epochs=100,
  		  batch_size=16384)

# Exercise III: Inspect the model (I)

Now that you've fit your model, let's take a look at it. You can use the `.get_weights()` method to inspect your model's weights.

The input layer will have 4 weights: 2 for each input times 2 for each output.

The output layer will have 2 weights, one for each output.

### Instructions

- Print the `model`'s weights.
- Print the column means of the training data (`games_tourney_train`).


In [ ]:
# Print the model's weights
print(model.get_weights())

# Print the column means of the training data
print(games_tourney_train.mean())

# Exercise IV: Evaluate the model

Now that you've fit your model and inspected it's weights to make sure it makes sense, evaluate it on the tournament test set to see how well it performs on new data.

### Instructions

- Evaluate the model on `games_tourney_test`.
- Use the same inputs and outputs as the training set.


In [ ]:
# Evaluate the model on the tournament test data
print(model.evaluate(games_tourney_test[['seed_diff', 'pred']], games_tourney_test[['score_1', 'score_2']], verbose=False))